# HOLA AMIGOS

In [1]:
% reset
from sklearn.model_selection import train_test_split
import numpy as np
from helpers import *
from cleaners import *
import os
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pathlib
from multiprocessing import Process,Lock,Value
%load_ext autoreload
%autoreload 2


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
dirs = ['aclImdb/train/pos','aclImdb/train/neg','aclImdb/train/unsup']
clean_root = speedy(pos_cleaner, dirs,15,target_suffix='pos')

cleaningaclImdb/train/unsup


In [3]:
from pathlib import Path
class MyDocs(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        pathlist = Path(self.dirname).glob('**/*')
        for path in pathlist:
            path_in_str = str(path)
            if os.path.isfile(path_in_str):
                f=open(path_in_str)
                yield f.read().split()

In [4]:
from gensim.models import Word2Vec,TfidfModel,Phrases
from gensim.corpora import Dictionary
import gensim

docs = MyDocs(clean_root)
bigram_transformer = Phrases(docs)
wv_model = Word2Vec(bigram_transformer[docs], size=300, window=5, min_count=100, workers=6)

/home/alyswidan/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [5]:
def tf_idf_model(doc_path,model_name=None):
    model_path = f'models/tf_{model_name}'
    dict_path = f'models/dict_{model_name}'
    model_exists = os.path.isfile(model_path)
    dict_exists = os.path.isfile(dict_path)
    dir_exists = os.path.isdir('models')
    save = model_name!=None
    
    docs = MyDocs(doc_path)
    bow_dict = Dictionary.load(dict_path) if dict_exists else Dictionary(docs)
    corpus = [bow_dict.doc2bow(doc) for doc in docs]
    tf_model = TfidfModel.load(model_path) if model_exists else TfidfModel(corpus)
    if save:
        if not dir_exists:
            pathlib.Path.mkdir('models')
        if not model_exists:
            tf_model.save(model_path)
        if not dict_exists:
            bow_dict.save(dict_path)
    return corpus,bow_dict,tf_model
        

pos_corpus, pos_dict, pos_tf_model = tf_idf_model(f'{clean_root}/train/pos','pos')
neg_corpus, neg_dict, neg_tf_model = tf_idf_model(f'{clean_root}/train/neg','neg')

In [73]:
doc0 = raw_docs[11]
bow0 = bows[11]
pos_tf_model[bow0]
tf_dict = {e0:e1 for e0,e1 in pos_tf_model[bow0]}
sorted_tfs = [tf_dict[tok_id] for tok_id in (pos_dict.token2id[tok] for tok in doc0 if tok in wv_model.wv.vocab)]
sorted_tfs

[0.021976807651603317,
 0.022925812720642178,
 0.070406607929402329,
 0.09951364440068855,
 0.050145578715350608,
 0.11540032501294552,
 0.10135678257370058,
 0.079098294430211241,
 0.1131214593145272,
 0.067350853376543673,
 0.098651363245904899,
 0.074836207845630845,
 0.071110856552041316,
 0.10750955500910521,
 0.10260015112089857,
 0.088945001706924334,
 0.10957172827794498,
 0.10993679309219441,
 0.092822766510904361,
 0.056974249706613633,
 0.055492060740391341,
 0.090862971171037865,
 0.033562197564937016,
 0.10316393397165001,
 0.11588855463830307,
 0.024087506740569888,
 0.097029654042302099,
 0.090296205570073701,
 0.10752524667042938,
 0.08308375299219975,
 0.12290430131149949,
 0.055693659906542618,
 0.10316393397165001,
 0.024935215704308161,
 0.076252663130828763,
 0.1131214593145272,
 0.074258213208723486,
 0.049040013306127464,
 0.053556465602493691,
 0.12435209915515089,
 0.067714281040856153,
 0.10752524667042938,
 0.062209120691583839,
 0.071110856552041316,
 0.1163

In [87]:
def doc2vec_factory(bow_dict,tf_model,word2vec):
    def doc2vec(doc_bow,raw_doc):
        tf_dict = {e0:e1 for e0,e1 in pos_tf_model[doc_bow]}
        doc_tf_idf = np.array([tf_dict[tok_id] for tok_id in (bow_dict.token2id[tok] for tok in raw_doc if tok in word2vec.wv.vocab)])
        vecs = np.array([word2vec.wv[word] for idx,word in enumerate(raw_doc) if word in word2vec.wv.vocab])
        doc_vec = np.sum(doc_tf_idf.reshape(-1,1)*vecs,axis=0)
        return doc_vec.reshape(-1,1)
    return doc2vec


In [88]:
pos_doc2vec = doc2vec_factory(pos_dict,pos_tf_model,wv_model)
neg_doc2vec = doc2vec_factory(neg_dict,neg_tf_model,wv_model)

pos_vecs = []
for bows,doc in zip(pos_corpus,MyDocs(f'{clean_root}/train/pos')):
    pos_vecs.append(pos_doc2vec(bows,doc))
pos_vecs = np.array(pos_vecs)


In [89]:
neg_vecs = []
for bows,doc in zip(neg_corpus,MyDocs(f'{clean_root}/train/neg')):
    neg_vecs.append(neg_doc2vec(bows,doc))
neg_vecs = np.array(neg_vecs)

In [90]:
neg_vecs = neg_vecs.reshape(-1,300)
pos_vecs = pos_vecs.reshape(-1,300)

In [91]:
X = np.concatenate((pos_vecs,neg_vecs),axis=0)
y = np.asarray([1]*pos_vecs.shape[0] + [-1]*pos_vecs.shape[0])

X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.33, random_state=42)


In [92]:
from sklearn.svm import SVC

clf = SVC(C=1)
clf.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [93]:
np.mean(clf.predict(X_val) == y_val)

0.88145454545454549

In [94]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=500,min_samples_split=50,n_jobs=-1)
forest.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=50,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [95]:
np.mean(forest.predict(X_val) == y_val)

0.83624242424242423

In [96]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=200)
ada.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=200, random_state=None)

In [97]:
np.mean(ada.predict(X_val) == y_val)

0.85224242424242425

In [98]:
from sklearn.ensemble import GradientBoostingClassifier
gboost = GradientBoostingClassifier(n_estimators=300)
gboost.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [99]:
np.mean(gboost.predict(X_val) == y_val)

0.86896969696969695